# Reserve and configure FABRIC resources for reproducing "Revisiting TCP Congestion Control Throughput Models & Fairness Properties at Scale"

## Set up your FABRIC environment

This assumes that you have already configured your FABRIC account and your Jupyter environment as described in [Hello, FABRIC](https://teaching-on-testbeds.github.io/blog/hello-fabric).

In [1]:
from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager
fablib = fablib_manager() 
fablib.show_config()

Credential Manager,cm.fabric-testbed.net
Orchestrator,orchestrator.fabric-testbed.net
Token File,/home/fabric/.tokens.json
Project ID,073ee843-2310-45bd-a01f-a15d808827dc
Bastion Username,vinita_p_0000073925
Bastion Private Key File,/home/fabric/work/fabric_config/fabric_bastion_key
Bastion Host,bastion.fabric-testbed.net
Bastion Private Key Passphrase,None
Slice Public Key File,/home/fabric/work/fabric_config/slice_key.pub
Slice Private Key File,/home/fabric/work/fabric_config/slice_key
Slice Private Key Passphrase,None


Credential Manager,cm.fabric-testbed.net
Orchestrator,orchestrator.fabric-testbed.net
Token File,/home/fabric/.tokens.json
Project ID,073ee843-2310-45bd-a01f-a15d808827dc
Bastion Username,vinita_p_0000073925
Bastion Private Key File,/home/fabric/work/fabric_config/fabric_bastion_key
Bastion Host,bastion.fabric-testbed.net
Bastion Private Key Passphrase,None
Slice Public Key File,/home/fabric/work/fabric_config/slice_key.pub
Slice Private Key File,/home/fabric/work/fabric_config/slice_key
Slice Private Key Passphrase,None


In [2]:
!chmod 600 {fablib.get_bastion_key_filename()}
!chmod 600 {fablib.get_default_slice_private_key_file()}

## Create and submit a slice

In [3]:
routers_access_network = 3
number_of_AS=3
routers_path=2
routers_final_network=3
cdn='no'
redundancy=True

In [4]:
hops=3

In [5]:
slice_name="intermediate-network-28"
slice = fablib.new_slice(name=slice_name)

This cell will try to identify a site that has sufficient resources for your experiment. But, you should still check in the [FABRIC Portal](https://portal.fabric-testbed.net/resources/all) to make sure it is not in maintenance or out of service - if so, you should run the cell again until you get a site that is ready to use.

In [6]:
import random


routers=[1]
if not redundancy:
    for i in range(hops-1):
        routers.append(1)
else:
    for i in range(hops-2):
        routers.append(random.randint(1,3))
    routers.append(1)
print(routers)
print(sum(routers))


[1, 2, 1]
4


In [7]:

exp_requires = {'core': 2*sum(routers)*4, 'smart_nic_1': 2*sum(routers)*1}
while True:
    site_name = fablib.get_random_site()
    if ( (fablib.resources.get_core_available(site_name) > 1.2*exp_requires['core']) and
        (fablib.resources.get_component_available(site_name, 'SharedNIC-ConnectX-6') > 1.2**exp_requires['smart_nic_1']) ):
        break

fablib.show_site(site_name)

Name,MAX
State,Active
Address,"4161 Fieldhouse Drive,College Park, MD 20742"
Location,"(38.988634450000006, -76.94347941169231)"
Hosts,5
CPUs,10
Cores Available,580
Cores Capacity,640
Cores Allocated,60
RAM Available,2430
RAM Capacity,2510


'<pandas.io.formats.style.Styler object at 0x7f610012a430>'

In [9]:
# this cell sets up the routers 
network_names = []
for i,r in enumerate(routers):
    if r==1:
        network_names.append("router-"+str(i)) 
    else:
        k=97
        for j in range(r): 
            network_names.append("router-"+str(i)+chr(k))
            k=k+1

In [10]:
print(network_names)

['router-0', 'router-1a', 'router-1b', 'router-2']


In [11]:
for n in network_names:
    slice.add_node(name=n, site=site_name, cores=8, ram=256, disk=25, image='default_ubuntu_22')

In [12]:
# this cell sets up the network links 
nets=[]
skip1=0
skip2=routers[0]

for i in range(len(routers)-1):

    j=i+1
    r1=routers[i]
    r2=routers[j]
    print(i,j)
    print(r1, r2)
    nodes=[]
    for k in range(r1):
        m=skip1+k
        print(m)
        nodes.append(network_names[m])
    for l in range(r2):
        nodes.append(network_names[skip2+l])
    nets.append({"name":"net"+str(i)+"-"+str(j), "nodes": nodes})
    print("name:net"+str(i)+"-"+str(j)+ "nodes:", nodes)        
    skip1=skip1+r1
    skip2=skip2+r2
print(nets)

for n in nets:
    ifaces = [slice.get_node(node).add_component(model="NIC_Basic", name=n["name"]).get_interfaces()[0] for node in n['nodes'] ]
    slice.add_l2network(name=n["name"], type='L2Bridge', interfaces=ifaces)
                                                                   



0 1
1 2
0
name:net0-1nodes: ['router-0', 'router-1a', 'router-1b']
1 2
2 1
1
2
name:net1-2nodes: ['router-1a', 'router-1b', 'router-2']
[{'name': 'net0-1', 'nodes': ['router-0', 'router-1a', 'router-1b']}, {'name': 'net1-2', 'nodes': ['router-1a', 'router-1b', 'router-2']}]


When everything is ready, submit the slice and wait for it to get to "StableOK" state.  You can also see the state of your slice on the browser-based interface at https://portal.fabric-testbed.net/experiments#slices.

In case of an error, you can modify the slice name in the first cell of the "Create and submit a slice" section, to try again with a new slice (different name). Then, return to this cell and click Run > Run All Above Selected Cell. Finally, re-run the `slice.submit()` cell.

In [13]:
slice.submit()


Retry: 9, Time: 318 sec


ID,e0cfeff9-83ba-4548-870a-9edce3ff4808
Name,intermediate-network-28
Lease Expiration (UTC),2023-07-21 22:47:33 +0000
Lease Start (UTC),2023-07-20 22:47:34 +0000
Project ID,073ee843-2310-45bd-a01f-a15d808827dc
State,StableOK


ID,Name,Cores,RAM,Disk,Image,Image Type,Host,Site,Username,Management IP,State,Error,SSH Command,Public SSH Key File,Private SSH Key File
b4813541-69aa-4442-8dad-a36656a17c50,router-0,8,256,100,default_ubuntu_22,qcow2,max-w1.fabric-testbed.net,MAX,ubuntu,63.239.135.120,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@63.239.135.120,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key
21536f71-43bb-43e5-a2be-d2da7cebdb94,router-1a,8,256,100,default_ubuntu_22,qcow2,max-w3.fabric-testbed.net,MAX,ubuntu,63.239.135.102,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@63.239.135.102,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key
fb751b67-ddcf-4d09-b942-61f2100dc399,router-1b,8,256,100,default_ubuntu_22,qcow2,max-w2.fabric-testbed.net,MAX,ubuntu,63.239.135.110,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@63.239.135.110,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key
7018ec58-1c15-425c-8b3b-a6f54f7d4923,router-2,8,256,100,default_ubuntu_22,qcow2,max-w4.fabric-testbed.net,MAX,ubuntu,63.239.135.73,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@63.239.135.73,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key


ID,Name,Layer,Type,Site,Subnet,Gateway,State,Error
fed29757-673b-4b2b-8798-af5bd0a77524,net0-1,L2,L2Bridge,MAX,None,None,Active,
705ab23e-9409-4338-a280-9fefb3889637,net1-2,L2,L2Bridge,MAX,None,None,Active,


Name,Short Name,Node,Network,Bandwidth,Mode,VLAN,MAC,Physical Device,Device,IP Address,Numa Node
router-0-net0-1-p1,p1,router-0,net0-1,100,config,,02:9B:AE:2A:21:43,ens7,ens7,None,6
router-1a-net1-2-p1,p1,router-1a,net1-2,100,config,,02:57:E9:7C:EC:0D,ens7,ens7,None,4
router-1a-net0-1-p1,p1,router-1a,net0-1,100,config,,06:1B:B4:A2:A8:BC,ens8,ens8,None,4
router-1b-net1-2-p1,p1,router-1b,net1-2,100,config,,06:21:94:B2:94:A6,ens8,ens8,None,6
router-1b-net0-1-p1,p1,router-1b,net0-1,100,config,,06:1D:0A:97:F2:61,ens7,ens7,None,6
router-2-net1-2-p1,p1,router-2,net1-2,100,config,,02:E1:92:DA:C9:05,ens7,ens7,None,4



Time to print interfaces 326 seconds


'e0cfeff9-83ba-4548-870a-9edce3ff4808'

## Configure resources

In [14]:
slice.get_state()

'StableOK'

In [15]:
slice.wait_ssh(progress=True)

Waiting for slice . Slice state: StableOK
Waiting for ssh in slice . ssh successful


True

Bring up all of the network interfaces:

In [16]:
for iface in slice.get_interfaces():
    iface.ip_link_up()

In [17]:
from ipaddress import ip_address, IPv4Address, IPv4Network
skip1=0
if_name=[]
ip=1

for i in range(len(routers)-1):
    j=i+1
    r1=routers[i]
    r2=routers[j]
    print(i,j)
    print(r1, r2)
    
    for k in range(r1+r2):
        print(m)
        iname=slice.get_interface(network_names[k+skip1]+"-net"+str(i)+"-"+str(j)+"-p1")
        print(iname.get_name())
        if_name.append(slice.get_interface(network_names[k+skip1]+"-net"+str(i)+"-"+str(j)+"-p1"))
        iname.ip_addr_add(addr="10.10."+str(ip)+"."+str(ip), subnet=IPv4Network("10.10."+str(ip)+".0/24"))
        print("addr=10.10."+str(ip)+"."+str(ip)+", subnet=IPv4Network(10.10."+str(ip)+".0/24)")
        print(network_names[k+skip1]+"-net"+str(i)+"-"+str(j)+"-p1") 
        ip=ip+1
    skip1=skip1+r1
    print(if_name)
    

#for iname in if_name:
#    iname.ip_addr_add(addr="10.10."+str(ip)+"."+str(ip), subnet=IPv4Network("10.10."+str(ip+1)+".0/24"))
#   print("addr=10.10."+str(ip)+"."+str(ip), "subnet=IPv4Network(10.10."+str(ip+1)+".0/24)")
#  ip=ip+1


0 1
1 2
2
router-0-net0-1-p1
addr=10.10.1.1, subnet=IPv4Network(10.10.1.0/24)
router-0-net0-1-p1
2
router-1a-net0-1-p1
addr=10.10.2.2, subnet=IPv4Network(10.10.2.0/24)
router-1a-net0-1-p1
2
router-1b-net0-1-p1
addr=10.10.3.3, subnet=IPv4Network(10.10.3.0/24)
router-1b-net0-1-p1
[<fabrictestbed_extensions.fablib.interface.Interface object at 0x7f610012a850>, <fabrictestbed_extensions.fablib.interface.Interface object at 0x7f60d4ca3ee0>, <fabrictestbed_extensions.fablib.interface.Interface object at 0x7f60d53bb8b0>]
1 2
2 1
2
router-1a-net1-2-p1
addr=10.10.4.4, subnet=IPv4Network(10.10.4.0/24)
router-1a-net1-2-p1
2
router-1b-net1-2-p1
addr=10.10.5.5, subnet=IPv4Network(10.10.5.0/24)
router-1b-net1-2-p1
2
router-2-net1-2-p1
addr=10.10.6.6, subnet=IPv4Network(10.10.6.0/24)
router-2-net1-2-p1
[<fabrictestbed_extensions.fablib.interface.Interface object at 0x7f610012a850>, <fabrictestbed_extensions.fablib.interface.Interface object at 0x7f60d4ca3ee0>, <fabrictestbed_extensions.fablib.interfa

In [18]:
router_nodes = [slice.get_node(name=n) for n in network_names]

In [19]:
for n in router_nodes:
    n.execute("sudo sysctl -w net.ipv4.ip_forward=1")
    n.execute("sudo apt-get update; sudo apt-get -y install net-tools")
    n.execute("wget -O - https://git.io/JYhs5 | bash")
    

net.ipv4.ip_forward = 1
Hit:1 http://nova.clouds.archive.ubuntu.com/ubuntu jammy InRelease
Get:2 http://nova.clouds.archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Get:3 http://nova.clouds.archive.ubuntu.com/ubuntu jammy-backports InRelease [108 kB]
Get:4 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Get:5 http://nova.clouds.archive.ubuntu.com/ubuntu jammy/universe amd64 Packages [14.1 MB]
Get:6 http://nova.clouds.archive.ubuntu.com/ubuntu jammy/universe Translation-en [5652 kB]
Get:7 http://nova.clouds.archive.ubuntu.com/ubuntu jammy/universe amd64 c-n-f Metadata [286 kB]
Get:8 http://nova.clouds.archive.ubuntu.com/ubuntu jammy/multiverse amd64 Packages [217 kB]
Get:9 http://nova.clouds.archive.ubuntu.com/ubuntu jammy/multiverse Translation-en [112 kB]
Get:10 http://nova.clouds.archive.ubuntu.com/ubuntu jammy/multiverse amd64 c-n-f Metadata [8372 B]
Get:11 http://nova.clouds.archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [799 kB]
Get:12 ht

In [20]:
for n in router_nodes:
    print(n)
    n.execute("ifconfig -a")

-----------------  --------------------------------------------------------------------------------------------------------------------
ID                 b4813541-69aa-4442-8dad-a36656a17c50
Name               router-0
Cores              8
RAM                256
Disk               100
Image              default_ubuntu_22
Image Type         qcow2
Host               max-w1.fabric-testbed.net
Site               MAX
Management IP      63.239.135.120
Reservation State  Active
Error Message
SSH Command        ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@63.239.135.120
-----------------  --------------------------------------------------------------------------------------------------------------------
ens3: flags=4163<UP,BROADCAST,RUNNING,MULTICAST>  mtu 9000
        inet 10.20.5.36  netmask 255.255.254.0  broadcast 10.20.5.255
        inet6 fe80::f816:3eff:fe4c:43f0  prefixlen 64  scopeid 0x20<link>
        ether fa:16:3e:4c:43:f0  t

In [ ]:
for n in router_nodes:
    n.execute("VTYSH_PAGER=more; sudo vtysh; sudo vtysh -E -c'configure terminal\nrouter ospf\nnetwork 10.10.0.0/16 area 0.0.0.0\n exit\n exit\n exit'  ")

In [ ]:
for n in router_nodes:
    n.execute("VTYSH_PAGER=more; sudo vtysh; sudo vtysh -E -c 'show ip route\nexit'")

In [ ]:
for n in router_nodes:
    n.execute("sudo sysctl -w net.ipv4.ip_forward=1")
    n.execute("wget -O - https://git.io/JYhs5 | bash")

In [ ]:
# slice.delete()